<a href="https://colab.research.google.com/github/VirGonzalez/IA_/blob/main/Ejemplo_Py_plan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U py_plan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 10.8 MB/s eta 0:00:00


In [ ]:
# Permite modelar operadores como funciones en python
from operator import ne

from py_search.uninformed import depth_first_search
from py_search.uninformed import breadth_first_search
from py_search.uninformed import iterative_deepening_search

from py_plan.total_order import StateSpacePlanningProblem
from py_plan.base import Operator

# Las acciones se definen como instancias de Operator
# Debemos indicar NOMBRE, CONDICIONES y EFECTOS (lista de tuplas)
# ?x : las variables se definen anteponiendo un ? al nombre
move = Operator('move',                      # nombre
                [('on', '?b', '?x'),         # condiciones
                 ('block', '?b'),
                 ('block', '?x'),
                 ('block', '?y'),
                 ('block', '?other'),
                 ('block', '?other2'),
                 ('not', ('on', '?other', '?b')),
                 ('not', ('on', '?other2', '?y')),
                 ('clear', '?b'),
                 ('clear', '?y'),
                 (ne, '?b', '?x'),            # ne representa la desigualdad
                 (ne, '?b', '?y'),
                 (ne, '?x', '?y')],
                [('on', '?b', '?y'),          # efectos
                 ('clear', '?x'),
                 ('not', ('on', '?b', '?x')),
                 ('not', ('clear', '?y'))
                 ])

move_from_table = Operator('move_from_table',
                           [('on', '?b', 'Table'),
                            ('block', '?other'),
                            ('block', '?other2'),
                            ('not', ('on', '?other', '?b')),
                            ('not', ('on', '?other2', '?y')),
                            ('clear', '?b'),
                            ('clear', '?y'),
                            ('block', '?b'),
                            ('block', '?y'),
                            (ne, '?b', '?y')],
                           [('on', '?b', '?y'),
                            ('not', ('on', '?b', 'Table')),
                            ('not', ('clear', '?y'))
                            ])

move_to_table = Operator('move_to_table',
                         [('on', '?b', '?x'),
                          ('block', '?b'),
                          ('block', '?x'),
                          ('block', '?other'),
                          ('not', ('on', '?other', '?b')),
                          ('clear', '?b'),
                          (ne, '?b', '?x')],
                         [('on', '?b', 'Table'),
                          ('clear', '?x'),
                          ('not', ('on', '?b', '?x'))])

# Indica que el código dentro del if es ejecutado cuando el programa es corrido directamente por el interprete
# No, cuando es importado como un módulo
if __name__ == "__main__":
    # Definición del ESTADO INICIAL (lista de tuplas)
    start = [('on', 'A', 'Table'),   # A sobre la mesa
             ('on', 'B', 'Table'),   # B sobre la mesa
             ('on', 'C', 'A'),       # C sobre A
             ('block', 'A'),         # A es un bloque
             ('block', 'B'),         # B es un bloque
             ('block', 'C'),         # C es un bloque
             ('clear', 'B'),         # B está libre
             ('clear', 'C')          # C está libre
             ]
     # Definición del ESTADO FINAL (lista de tuplas)
    goal = [('on', 'A', 'B'),        # A sobre B
            ('on', 'B', 'C'),        # B sobre C
            ('on', 'C', 'Table')]    # C sobre la mesa

    # Progresión, busqueda hacia adelante
    def progression(x):
        return breadth_first_search(x, forward=True, backward=False)

    # Definicion del problema con Estado Inicial, Estado Final y Acciones
    p = StateSpacePlanningProblem(start, goal, [move_from_table,
                                                move_to_table, move])

    # Ejecución del planner y visualización de resultados
    print(next(progression(p)).path())

    # Ejecución del planner y visualización de resultados con Busqueda iterativa en profundidad
    print(next(iterative_deepening_search(p)).path())

((move_to_table, {'?b': 'C', '?x': 'A', '?other': 'B'}), (move_from_table, {'?other': 'C', '?y': 'C', '?other2': 'C', '?b': 'B'}), (move_from_table, {'?other': 'C', '?b': 'A', '?other2': 'C', '?y': 'B'}))
((move_to_table, {'?b': 'C', '?x': 'A', '?other': 'A'}), (move_from_table, {'?other': 'A', '?y': 'C', '?other2': 'C', '?b': 'B'}), (move_from_table, {'?other2': 'B', '?other': 'A', '?b': 'A', '?y': 'B'}))
